In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from base.dataset import ABAW2_VA_Arranger, ABAW2_VA_Dataset
from model.model import my_2d1ddy
import os
import torch.nn as nn
from tqdm import tqdm
from base.logger import ContinuousOutputHandlerNPYTrial, ContinuousMetricsCalculatorTrial

import pandas as pd
import math
import openpyxl

batch_size = 16
window_length = 300
hop_length = 200
debug = 0
fold = 0
time_delay = 0
train_emotion = 'both'
modality = ['frame', 'mfcc', 'vggish']
head = "multi-headed"

backbone_state_dict = "res50_ir_0.887"
backbone_mode = "ir"
cnn1d_embedding_dim = 512
cnn1d_channels = [128,128,128,128]
output_dim = 2
cnn1d_kernel_size = 5
cnn1d_attention = 0
cnn1d_dropout = 0.1
model_load_path = "load"

root_path = "save"
model_state_dict_path_list = []

class CCCLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, gold, pred, weights=None):
        # pred = torch.tanh(pred)
        gold_mean = torch.mean(gold, 1, keepdim=True, out=None)
        pred_mean = torch.mean(pred, 1, keepdim=True, out=None)
        covariance = (gold - gold_mean) * (pred - pred_mean)
        gold_var = torch.var(gold, 1, keepdim=True, unbiased=True, out=None)
        pred_var = torch.var(pred, 1, keepdim=True, unbiased=True, out=None)
        ccc = 2. * covariance / (
                (gold_var + pred_var + torch.mul(gold_mean - pred_mean, gold_mean - pred_mean)) + 1e-08)
        ccc_loss = 1. - ccc

        if weights is not None:
            ccc_loss *= weights

        return torch.mean(ccc_loss)
    
def get_train_emotion(emotion_tag, head):

    emotion = ["Valence", "Arousal"]

    if emotion_tag == "arousal":
        if head == "sh":
            emotion = ["Arousal"]
    elif emotion_tag == "valence":
        if head == "sh":
            emotion = ["Valence"]

    return emotion

from tqdm import tqdm
from base.logger import ContinuousOutputHandlerNPYTrial, ContinuousMetricsCalculatorTrial


if not torch.cuda.is_available():
    device = torch.device('cpu')
else:
    device = torch.device('cuda')
    
emotional_dimension = get_train_emotion(train_emotion, head)
metrics = ['ccc']
    

criterion = CCCLoss()

trials_pred = {}

loss_dict = {}

def loop(data_loader, train_mode=False):
    output_handler = ContinuousOutputHandlerNPYTrial(emotional_dimension)
    continuous_label_handler = ContinuousOutputHandlerNPYTrial(emotional_dimension)    
    
    metric_handler = ContinuousMetricsCalculatorTrial(metrics, emotional_dimension,
                                                        output_handler, continuous_label_handler)
    
    total_batch_counter = 0
    for batch_index, (X, Y, trials, lengths, indices) in tqdm(enumerate(data_loader), total=len(data_loader)):
        total_batch_counter += len(trials)    
    
        if 'frame' in X:
            inputs = X['frame'].to(device)

        if 'mfcc' in X:
            inputs3 = X['mfcc'].to(device)

        if 'vggish' in X:
            inputs4 = X['vggish'].to(device)

        labels = Y.float().to(device)
        loss_weights = torch.ones_like(labels).to(device)
        
        outputs = model(inputs, inputs4, inputs3)
        
        output_handler.update_output_for_seen_trials(outputs.detach().cpu().numpy(), trials, indices, lengths)
        continuous_label_handler.update_output_for_seen_trials(labels.detach().cpu().numpy(), trials, indices, lengths)
        
        
        loss = criterion(outputs, labels, loss_weights)
        
        for i in range(len(trials)):
            try:
                trials_pred[trials[i]] = torch.cat([trials_pred[trials[i]], outputs[i]], dim=0)
                loss_dict[trials[i]].append(loss)
            except:
                trials_pred[trials[i]] = outputs[i]
                loss_dict[trials[i]] = [loss]
            
    output_handler.average_trial_wise_records()
    continuous_label_handler.average_trial_wise_records()

    output_handler.concat_records()
    continuous_label_handler.concat_records()
    
    metric_handler.calculate_metrics()
    
    return outputs, trials_pred, loss_dict, output_handler, continuous_label_handler, metric_handler
    

for i in os.listdir(root_path):
    if i.__contains__("pre") or i .__contains__("results"): continue
    model_dir_path = os.path.join(root_path, i, "0")
    model_path = os.path.join(model_dir_path, "model_state_dict.pth")
    model_state_dict_path_list.append(model_path)


from datetime import datetime

xl_dir_path = "xl_dir"
if not os.path.exists(xl_dir_path):
    os.makedirs(xl_dir_path)

init_time = datetime.now()
init_time = init_time.strftime('%m%d_%H%M')
csv_name = f"{xl_dir_path}/{init_time}_test_score.csv"

columns = ["Seed", "CCC_Valence", "CCC_Arousal", "CCC_Mean"]
init_df = pd.DataFrame(columns=columns)

init_df.to_csv(csv_name, index=False)
file_values_dict = {c:[] for c in columns}

In [8]:
##############################################
for seed in range(10):
    model_state_dict_path = model_state_dict_path_list[seed]
    model_state_dict = torch.load(model_state_dict_path)

    for key in list(model_state_dict.keys()):
        if 'module.' in key:
            model_state_dict[key.replace('module.','')] = model_state_dict[key]
            del model_state_dict[key]
            
    model = my_2d1ddy(backbone_state_dict=backbone_state_dict, backbone_mode=backbone_mode,
                                        embedding_dim=cnn1d_embedding_dim, channels=cnn1d_channels, modality=modality,
                                        output_dim=output_dim, kernel_size=cnn1d_kernel_size, attention=cnn1d_attention,
                                        dropout=cnn1d_dropout, root_dir=model_load_path)
    model.init()
    model.load_state_dict(model_state_dict)
    
    # dataset_path = "../data/Affwild2/seed_data.json"
    dataset_path = "Affwild2_processed_ver3"

    arranger = ABAW2_VA_Arranger(dataset_path, window_length=window_length, hop_length=hop_length,
                                debug=debug)

    mean_std_info = arranger.mean_std_info
    data_dict = arranger.resample_according_to_window_and_hop_length(seed)

    test_dataset = ABAW2_VA_Dataset(data_dict['Test_Set'], time_delay=time_delay,
                                        emotion=train_emotion, modality=modality,
                                        head=head, mode='validate', fold=fold, mean_std_info=mean_std_info)
    test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset, batch_size=batch_size, shuffle=False)
    
    model.cuda(device=device)
    with torch.no_grad():
        model.eval()
        outputs, trials_pred, loss, output_handler, continuous_label_handler, metric_handler = loop(test_loader, train_mode=False)
        
    metric_handler.calculate_metrics()
    ccc_result = metric_handler.metric_record_dict
    new_ccc_result = {} # {"vid" : [valence, arousal, mean]}
    for k in ccc_result.keys():
        new_ccc_result[k] = [ccc_result[k]['Valence']['ccc'][0], ccc_result[k]['Arousal']['ccc'][0], np.mean([ccc_result[k]['Valence']['ccc'][0], ccc_result[k]['Arousal']['ccc'][0]])]
        
    sort_vid = sorted(new_ccc_result.items(), key=lambda x: x[1][2])
    
    print("before sort_vid len : ", len(sort_vid))
    for idx, i in enumerate(sort_vid):
        if i[0] == 'overall':
            del sort_vid[idx]
            
    print("after sort_vid len : ", len(sort_vid))
    
    valence = []
    arousal = []
    mean = []

    for vid,score_list in sort_vid:    
        valence.append(score_list[0])
        arousal.append(score_list[1])
        mean.append(score_list[2])
        
    val_std = np.round(np.std(valence), 3)
    aro_std = np.round(np.std(arousal), 3)
    mean_std = np.round(np.std(mean), 3)
    
    file_values_dict['Seed'].append(seed)
    file_values_dict['CCC_Valence'].append(str(np.round(np.mean(valence), 3)) + "±" + str(val_std))
    file_values_dict['CCC_Arousal'].append(str(np.round(np.mean(arousal), 3)) + "±" + str(aro_std))
    file_values_dict['CCC_Mean'].append(str(np.round(np.mean(mean), 3)) + "±" + str(mean_std))
    
    print(file_values_dict['CCC_Valence'])
    print(file_values_dict['CCC_Arousal'])
    print(file_values_dict['CCC_Mean'])
        
full_record_df = pd.DataFrame(file_values_dict, columns=columns)
full_record_df.to_csv(csv_name,  mode='a', header=False, index=False, encoding='utf-8')

100%|██████████| 104/104 [21:14<00:00, 12.26s/it]


before sort_vid len :  84
after sort_vid len :  83
['0.405±0.288']
['0.454±0.233']
['0.43±0.218']


100%|██████████| 122/122 [25:46<00:00, 12.67s/it]


before sort_vid len :  84
after sort_vid len :  83
['0.405±0.288', '0.402±0.228']
['0.454±0.233', '0.442±0.234']
['0.43±0.218', '0.422±0.198']


100%|██████████| 133/133 [26:25<00:00, 11.92s/it]


...

before sort_vid len :  84
after sort_vid len :  83
['0.405±0.288', '0.402±0.228', '0.389±0.242', '0.343±0.255', '0.392±0.287', '0.38±0.256', '0.383±0.264', '0.347±0.276']
['0.454±0.233', '0.442±0.234', '0.441±0.219', '0.417±0.269', '0.453±0.248', '0.432±0.238', '0.402±0.231', '0.414±0.229']
['0.43±0.218', '0.422±0.198', '0.415±0.196', '0.38±0.235', '0.423±0.225', '0.406±0.217', '0.392±0.221', '0.381±0.212']


100%|██████████| 118/118 [22:54<00:00, 11.65s/it]


before sort_vid len :  84
after sort_vid len :  83
['0.405±0.288', '0.402±0.228', '0.389±0.242', '0.343±0.255', '0.392±0.287', '0.38±0.256', '0.383±0.264', '0.347±0.276', '0.366±0.24']
['0.454±0.233', '0.442±0.234', '0.441±0.219', '0.417±0.269', '0.453±0.248', '0.432±0.238', '0.402±0.231', '0.414±0.229', '0.412±0.24']
['0.43±0.218', '0.422±0.198', '0.415±0.196', '0.38±0.235', '0.423±0.225', '0.406±0.217', '0.392±0.221', '0.381±0.212', '0.389±0.206']


100%|██████████| 115/115 [26:17<00:00, 13.72s/it]


before sort_vid len :  84
after sort_vid len :  83
['0.405±0.288', '0.402±0.228', '0.389±0.242', '0.343±0.255', '0.392±0.287', '0.38±0.256', '0.383±0.264', '0.347±0.276', '0.366±0.24', '0.417±0.282']
['0.454±0.233', '0.442±0.234', '0.441±0.219', '0.417±0.269', '0.453±0.248', '0.432±0.238', '0.402±0.231', '0.414±0.229', '0.412±0.24', '0.435±0.245']
['0.43±0.218', '0.422±0.198', '0.415±0.196', '0.38±0.235', '0.423±0.225', '0.406±0.217', '0.392±0.221', '0.381±0.212', '0.389±0.206', '0.426±0.219']


In [15]:
char = "±"

valence = []
arousal = []
mean = []

for v, a, m in zip(file_values_dict['CCC_Valence'], file_values_dict['CCC_Arousal'], file_values_dict['CCC_Mean']):
    v,a,m = float(v.split(char)[0]), float(a.split(char)[0]), float(m.split(char)[0])
    valence.append(v)
    arousal.append(a)
    mean.append(m)
    
val_std =  np.round(np.std(valence), 3) 
aro_std =  np.round(np.std(arousal), 3)
mean_std = np.round(np.std(mean), 3)

val_mean = np.round(np.average(valence), 3) 
aro_mean = np.round(np.average(arousal), 3) 
mean_mean = np.round(np.average(mean), 3) 

print("val_mean : ", val_mean)
print("aro_mean : ", aro_mean)
print("mean_mean : ", mean_mean)
print("val_std : ", val_std)
print("aro_std : ", aro_std)
print("mean_std : ", mean_std)

val_mean :  0.382
aro_mean :  0.43
mean_mean :  0.406
val_std :  0.023
aro_std :  0.017
mean_std :  0.018


: 